In [4]:
import pandas as pd
import numpy as np
import re
import gc 
import os


In [ ]:

player_injuries = pd.read_csv("Datasets/player_injuries.csv", low_memory=False)
player_latest_market_value = pd.read_csv("Datasets/player_latest_market_value.csv", low_memory=False)
player_market_value = pd.read_csv("Datasets/player_market_value.csv", low_memory=False)
player_national_performances = pd.read_csv("Datasets/player_national_performances.csv", low_memory=False)
player_performances = pd.read_csv("Datasets/player_performances.csv", low_memory=False)
player_profiles = pd.read_csv("Datasets/player_profiles.csv", low_memory=False)
player_teammates_played_with = pd.read_csv("Datasets/player_teammates_played_with.csv", low_memory=False)
team_children = pd.read_csv("Datasets/team_children.csv", low_memory=False)
team_competitions_seasons = pd.read_csv("Datasets/team_competitions_seasons.csv", low_memory=False)
team_details = pd.read_csv("Datasets/team_details.csv", low_memory=False)
transfer_history = pd.read_csv("Datasets/transfer_history.csv", low_memory=False)

print("All datasets loaded successfully!")

All datasets loaded successfully!


In [3]:
print(player_injuries.columns.tolist())
print(player_latest_market_value.columns.tolist())
print(player_market_value.columns.tolist())
print(player_national_performances.columns.tolist())
print(player_performances.columns.tolist())
print(player_profiles.columns.tolist())
print(player_teammates_played_with.columns.tolist())
print(team_children.columns.tolist())
print(team_competitions_seasons.columns.tolist())
print(team_details.columns.tolist())
print(transfer_history.columns.tolist())
print(player_latest_market_value.columns.tolist())


['player_id', 'season_name', 'injury_reason', 'from_date', 'end_date', 'days_missed', 'games_missed']
['player_id', 'date_unix', 'value']
['player_id', 'date_unix', 'value']
['player_id', 'team_id', 'matches', 'goals', 'shirt_number', 'debut', 'coach_id', 'debut_game_id', 'career_state']
['player_id', 'season_name', 'competition_id', 'competition_name', 'team_id', 'team_name', 'nb_in_group', 'nb_on_pitch', 'goals', 'assists', 'own_goals', 'subed_in', 'subed_out', 'yellow_cards', 'second_yellow_cards', 'direct_red_cards', 'penalty_goals', 'minutes_played', 'goals_conceded', 'clean_sheets']
['player_id', 'player_slug', 'player_name', 'player_image_url', 'name_in_home_country', 'date_of_birth', 'place_of_birth', 'country_of_birth', 'height', 'citizenship', 'is_eu', 'position', 'main_position', 'foot', 'current_club_id', 'current_club_name', 'joined', 'contract_expires', 'outfitter', 'social_media_url', 'player_agent_id', 'player_agent_name', 'contract_option', 'date_of_last_contract_exten

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Unique Player Count
n_profiles = player_profiles['player_id'].nunique()
n_market = player_market_value['player_id'].nunique()
n_perf = player_performances['player_id'].nunique()

print(f" UNIQUE PLAYERS FOUND:")
print(f" - In Profiles:      {n_profiles:,}")
print(f" - In Market Value:  {n_market:,} (Target Variable)")
print(f" - In Performances:  {n_perf:,}")

# Mismatch and URL columns checks
missing_profiles = n_market - n_profiles
if missing_profiles > 0:
    print(f" Warning: {missing_profiles} players have market values but NO profile data.")
else:
    print(" All players with market values have profile data.")

print("\n URL COLUMN ANALYSIS:")
url_cols = ['player_image_url', 'social_media_url', 'second_club_url', 'third_club_url']

# Calculate how many are missing 
for col in url_cols:
    if col in player_profiles.columns:
        missing_pct = player_profiles[col].isnull().mean() * 100
        present_pct = 100 - missing_pct
        print(f" - {col}: {present_pct:.1f}% have data, {missing_pct:.1f}% are empty.")

 UNIQUE PLAYERS FOUND:
 - In Profiles:      92,671
 - In Market Value:  69,441 (Target Variable)
 - In Performances:  88,375
 All players with market values have profile data.

🔗 URL COLUMN ANALYSIS:
 - player_image_url: 100.0% have data, 0.0% are empty.
 - social_media_url: 21.9% have data, 78.1% are empty.
 - second_club_url: 1.1% have data, 98.9% are empty.
 - third_club_url: 0.0% have data, 100.0% are empty.


In [3]:
dfs = {
"player_injuries": player_injuries,
"player_latest_market_value": player_latest_market_value,
"player_market_value": player_market_value,
"player_national_performances": player_national_performances,
"player_performances": player_performances,
"player_profiles": player_profiles,
"player_teammates_played_with": player_teammates_played_with,
"team_children": team_children,
"team_competitions_seasons": team_competitions_seasons,
"team_details": team_details,
"transfer_history": transfer_history
}

summary = {}
for name, df in dfs.items():
    url_cols = [c for c in df.columns if any(k in c.lower() for k in ['url','website','logo'])]
    dropped = []
    indicators = []
    for c in url_cols:
        p = df[c].notna().mean()
        if p == 1.0 or p == 0.0:
            df.drop(columns=c, inplace=True, errors='ignore')
            dropped.append(c)
        else:
            ind = c + "_has_url"
            df[ind] = df[c].notna().astype('int8')
            df.drop(columns=c, inplace=True, errors='ignore')
            indicators.append(ind)
    summary[name] = {"url_columns_found": url_cols, "dropped": dropped, "indicators_created": indicators, "remaining_url_like": [c for c in df.columns if any(k in c.lower() for k in ['url','website','logo'])]}
    globals()[name] = df

for k,v in summary.items():
    print(f"{k}: found={len(v['url_columns_found'])}, dropped={len(v['dropped'])}, indicators={len(v['indicators_created'])}, remaining_url_like={v['remaining_url_like']}")


player_injuries: found=0, dropped=0, indicators=0, remaining_url_like=[]
player_latest_market_value: found=0, dropped=0, indicators=0, remaining_url_like=[]
player_market_value: found=0, dropped=0, indicators=0, remaining_url_like=[]
player_national_performances: found=0, dropped=0, indicators=0, remaining_url_like=[]
player_performances: found=0, dropped=0, indicators=0, remaining_url_like=[]
player_profiles: found=5, dropped=1, indicators=4, remaining_url_like=['social_media_url_has_url', 'second_club_url_has_url', 'third_club_url_has_url', 'fourth_club_url_has_url']
player_teammates_played_with: found=0, dropped=0, indicators=0, remaining_url_like=[]
team_children: found=0, dropped=0, indicators=0, remaining_url_like=[]
team_competitions_seasons: found=0, dropped=0, indicators=0, remaining_url_like=[]
team_details: found=2, dropped=2, indicators=0, remaining_url_like=[]
transfer_history: found=0, dropped=0, indicators=0, remaining_url_like=[]


In [4]:
for name, df in dfs.items():
    print(f"{name:35} → rows: {len(df):10} | cols: {df.shape[1]}")


player_injuries                     → rows:     143195 | cols: 7
player_latest_market_value          → rows:      69441 | cols: 3
player_market_value                 → rows:     901429 | cols: 3
player_national_performances        → rows:      92701 | cols: 9
player_performances                 → rows:    1878719 | cols: 20
player_profiles                     → rows:      92671 | cols: 33
player_teammates_played_with        → rows:    1257342 | cols: 6
team_children                       → rows:       7695 | cols: 5
team_competitions_seasons           → rows:     196378 | cols: 7
team_details                        → rows:       2175 | cols: 10
transfer_history                    → rows:    1101440 | cols: 10


In [5]:
for name, df in dfs.items():
    print("\n---", name, "---")
    na = (df.isna().mean()*100).sort_values(ascending=False)
    display(na[na>0].head(20))



--- player_injuries ---


end_date       1.063585
from_date      0.015364
days_missed    0.015364
dtype: float64


--- player_latest_market_value ---


Series([], dtype: float64)


--- player_market_value ---


...

club_division       16.391347
competition_id      16.391347
competition_name    16.391347
dtype: float64


--- team_details ---


club_division       23.264368
competition_id      23.264368
competition_name    23.264368
competition_slug    23.264368
dtype: float64


--- transfer_history ---


transfer_date    0.052658
season_name      0.000182
dtype: float64

In [6]:
for name, df in dfs.items():
    print("\n---", name, "---")
    display(df.dtypes.value_counts())



--- player_injuries ---


object     4
int64      2
float64    1
Name: count, dtype: int64


--- player_latest_market_value ---


int64      1
object     1
float64    1
Name: count, dtype: int64


--- player_market_value ---


...

object    5
int64     2
Name: count, dtype: int64


--- team_details ---


object    8
int64     2
Name: count, dtype: int64


--- transfer_history ---


int64     5
object    5
Name: count, dtype: int64

In [ ]:
DTYPES = { 'player_id': 'int32', 'team_id': 'Int32', 
          'goals': 'float32', 'assists': 'float32', 'minutes_played': 'float32', 'value': 'float32', 
          'transfer_fee': 'float32', 'height': 'float32', 'days_missed': 'float32', 'games_missed': 'float32' }

URL_COLS = [ 'player_image_url', 'social_media_url', 'second_club_url', 'third_club_url', 'fourth_club_url' ]
DROP_COLS = [ 'player_slug', 'name_in_home_country', 'place_of_birth', 'outfitter', 'player_agent_name', 'source_url' ] 


def clean_dataframe(df, name):
    print(f"\n--- Cleaning {name} ---")
    
    # dtype conversion
    for col, dtype in DTYPES.items():
        if col in df.columns:
            try:
                df[col] = df[col].astype(dtype)
            except:
                pass
    
    #  Handle URL → binary + drop
    url_in_df = [c for c in URL_COLS if c in df.columns]
    for c in url_in_df:
        df[c + "_has_url"] = df[c].notna().astype('int8')
        df.drop(columns=c, inplace=True)
    
    # Drop pre-defined low-info columns
    drop_in_df = [c for c in DROP_COLS if c in df.columns]
    df.drop(columns=drop_in_df, inplace=True)
    
    #  Drop columns with >90% missing
    na_percent = df.isna().mean()
    high_na_cols = na_percent[na_percent > 0.90].index.tolist()
    df.drop(columns=high_na_cols, inplace=True)
    
    #  Report
    print(f"  URL converted: {url_in_df}")
    print(f"  Dropped low-info cols: {drop_in_df}")
    print(f"  Dropped >90% NA cols: {high_na_cols}")
    print(f"  Final shape: {df.shape}")
    
    return df


player_injuries = clean_dataframe(player_injuries, "player_injuries")
player_latest_market_value = clean_dataframe(player_latest_market_value, "player_latest_market_value")
player_market_value = clean_dataframe(player_market_value, "player_market_value")
player_national_performances = clean_dataframe(player_national_performances, "player_national_performances")
player_performances = clean_dataframe(player_performances, "player_performances")
player_profiles = clean_dataframe(player_profiles, "player_profiles")
player_teammates_played_with = clean_dataframe(player_teammates_played_with, "player_teammates_played_with")
team_children = clean_dataframe(team_children, "team_children")
team_competitions_seasons = clean_dataframe(team_competitions_seasons, "team_competitions_seasons")
team_details = clean_dataframe(team_details, "team_details")
transfer_history = clean_dataframe(transfer_history, "transfer_history")

print("\n\n✅ ALL DATAFRAMES CLEANED AND READY FOR AGGREGATION!")



--- Cleaning player_injuries ---
  URL converted: []
  Dropped low-info cols: []
  Dropped >90% NA cols: []
  Final shape: (143195, 7)

--- Cleaning player_latest_market_value ---
  URL converted: []
  Dropped low-info cols: []
  Dropped >90% NA cols: []
  Final shape: (69441, 3)

--- Cleaning player_market_value ---
  URL converted: []
  Dropped low-info cols: []
  Dropped >90% NA cols: []
  Final shape: (901429, 3)

--- Cleaning player_national_performances ---
  URL converted: []
  Dropped low-info cols: []
  Dropped >90% NA cols: ['debut']
  Final shape: (92701, 8)

--- Cleaning player_performances ---
  URL converted: []
  Dropped low-info cols: []
  Dropped >90% NA cols: []
  Final shape: (1878719, 20)

--- Cleaning player_profiles ---
  URL converted: []
  Dropped low-info cols: ['player_slug', 'name_in_home_country', 'place_of_birth', 'outfitter', 'player_agent_name']
  Dropped >90% NA cols: ['contract_option', 'date_of_last_contract_extension', 'on_loan_from_club_id', 'on_loa

In [9]:
player_profiles.columns.tolist()
player_performances.columns.tolist()
player_injuries.columns.tolist()


['player_id',
 'season_name',
 'injury_reason',
 'from_date',
 'end_date',
 'days_missed',
 'games_missed']

In [ ]:
#Import Helper Functions
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', 200)
today = pd.to_datetime("today").normalize()

def normalize_season(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip()
    if "/" in s:
        p = s.split("/")
        try:
            if len(p[0])==2:
                y = int("20"+p[0]) if int(p[0])<=30 else int("19"+p[0])
                return y
            return int(p[0])
        except:
            return np.nan
    if "-" in s:
        try:
            return int(s.split("-")[0])
        except:
            return np.nan
    if s.isdigit() and len(s)==4:
        return int(s)
    return np.nan


In [ ]:
player_profiles['player_id'] = player_profiles['player_id'].astype(str)
player_injuries['player_id'] = player_injuries['player_id'].astype(str)
player_performances['player_id'] = player_performances['player_id'].astype(str)
player_national_performances['player_id'] = player_national_performances['player_id'].astype(str)
player_market_value['player_id'] = player_market_value['player_id'].astype(str)
player_latest_market_value['player_id'] = player_latest_market_value['player_id'].astype(str)
player_teammates_played_with['player_id'] = player_teammates_played_with['player_id'].astype(str)
transfer_history['player_id'] = transfer_history['player_id'].astype(str)

for df,cols in [
    (player_profiles, ['date_of_birth','joined','contract_expires','date_of_last_contract_extension','contract_there_expires']),
    (player_injuries, ['from_date','end_date']),
    (transfer_history, ['transfer_date']),
    (player_market_value, ['date_unix']),
    (player_latest_market_value, ['date_unix'])
]:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], dayfirst=True, errors='coerce')

if 'season_name' in player_injuries.columns:
    player_injuries['season_norm'] = player_injuries['season_name'].apply(normalize_season)
if 'season_name' in player_performances.columns:
    player_performances['season_norm'] = player_performances['season_name'].apply(normalize_season)
if 'season_name' in player_national_performances.columns:
    player_national_performances['season_norm'] = player_national_performances['season_name'].apply(normalize_season)
if 'season_id' in team_competitions_seasons.columns:
    team_competitions_seasons['season_norm'] = team_competitions_seasons['season_id'].astype(str).apply(normalize_season)


C:\Users\karan\AppData\Local\Temp\ipykernel_27228\1288594709.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], dayfirst=True, errors='coerce')
C:\Users\karan\AppData\Local\Temp\ipykernel_27228\1288594709.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], dayfirst=True, errors='coerce')
C:\Users\karan\AppData\Local\Temp\ipykernel_27228\1288594709.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], dayfirst=True, errors='coerce')
C:\Users\karan\AppData\Local\Temp\ipykernel_27228\1288594709.py:19: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=

In [ ]:
#Aggregate Injury Data
if 'days_missed' in player_injuries.columns:
    player_injuries['days_missed'] = pd.to_numeric(player_injuries['days_missed'], errors='coerce').fillna(0)
inj_agg = player_injuries.groupby('player_id').agg(
    total_injuries = ('injury_reason','count'),
    total_days_out = ('days_missed','sum'),
    mean_days_out = ('days_missed','mean'),
    seasons_with_injury = ('season_norm', lambda x: x.nunique())
).fillna(0)
inj_agg = inj_agg.astype({'total_injuries':'int32','seasons_with_injury':'int32'})


In [ ]:
#Aggregate Performance Data
perf = player_performances.copy()
num_cols = [c for c in ['minutes_played','minutes','goals','assists','shots','key_passes','dribbles','tackles','clean_sheets','goals_conceded'] if c in perf.columns]
for c in num_cols:
    perf[c] = pd.to_numeric(perf[c], errors='coerce').fillna(0)
agg_dict = {c:'sum' for c in num_cols}
perf_agg = perf.groupby('player_id').agg(agg_dict)
if 'minutes_played' in perf_agg.columns:
    perf_agg['goals_per90'] = perf_agg.get('goals',0) / (perf_agg['minutes_played']/90).replace(0,np.nan)
    perf_agg['assists_per90'] = perf_agg.get('assists',0) / (perf_agg['minutes_played']/90).replace(0,np.nan)
perf_agg = perf_agg.fillna(0)


In [ ]:
#Aggregate National Performance Data
nat = player_national_performances.copy()
nat_cols = [c for c in ['matches','goals','assists','minutes_played','avg_rating'] if c in nat.columns]
for c in nat_cols:
    nat[c] = pd.to_numeric(nat[c], errors='coerce').fillna(0)
nat_agg = nat.groupby('player_id').agg({c:'sum' for c in nat_cols})
if 'minutes_played' in nat_agg.columns:
    nat_agg['nat_goals_per90'] = nat_agg.get('goals',0) / (nat_agg['minutes_played']/90).replace(0,np.nan)
nat_agg = nat_agg.fillna(0)


In [ ]:
#Historical +last market value aggregation
pv = player_market_value.copy()
if 'date_unix' in pv.columns:
    pv['date_unix'] = pd.to_datetime(pv['date_unix'], dayfirst=True, errors='coerce')
pv['value'] = pd.to_numeric(pv['value'], errors='coerce').fillna(0)
pv_sorted = pv.sort_values(['player_id','date_unix'])
pv_agg = pv_sorted.groupby('player_id').agg(
    mv_max = ('value','max'),
    mv_min = ('value','min'),
    mv_mean = ('value','mean'),
    mv_std = ('value','std'),
    mv_count = ('value','count')
).fillna(0)
def growth(g):
    if len(g) < 2:
        return 0.0
    first = g['value'].iloc[0]
    last = g['value'].iloc[-1]
    if first == 0:
        return 0.0
    return (last - first) / first
pv_growth = pv_sorted.groupby('player_id').apply(growth).rename('mv_growth')
pv_agg = pv_agg.join(pv_growth).fillna(0)

pv_latest = player_latest_market_value.copy()
if 'date_unix' in pv_latest.columns:
    pv_latest['date_unix'] = pd.to_datetime(pv_latest['date_unix'], dayfirst=True, errors='coerce')
pv_latest['value'] = pd.to_numeric(pv_latest['value'], errors='coerce').fillna(0)
pv_latest_agg = pv_latest.groupby('player_id').agg(mv_current = ('value','last')).fillna(0)
pv_agg = pv_agg.join(pv_latest_agg).fillna(0)


C:\Users\karan\AppData\Local\Temp\ipykernel_27228\3000236171.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pv_growth = pv_sorted.groupby('player_id').apply(growth).rename('mv_growth')


In [24]:
tr = transfer_history.copy()
tr['player_id'] = tr['player_id'].astype(str)
if 'transfer_date' in tr.columns:
    tr['transfer_date'] = pd.to_datetime(tr['transfer_date'], dayfirst=True, errors='coerce')
fee_col = 'transfer_fee' if 'transfer_fee' in tr.columns else ('value_at_transfer' if 'value_at_transfer' in tr.columns else None)
if fee_col:
    tr['transfer_fee_norm'] = pd.to_numeric(tr[fee_col], errors='coerce')
else:
    tr['transfer_fee_norm'] = 0
tr_agg = tr.groupby('player_id').agg(
    num_transfers = ('transfer_date','count'),
    total_transfer_fees = ('transfer_fee_norm','sum'),
    last_transfer_fee = ('transfer_fee_norm','max'),
    first_transfer_date = ('transfer_date','min'),
    last_transfer_date = ('transfer_date','max')
)
tr_agg['last_transfer_date'] = pd.to_datetime(tr_agg['last_transfer_date'], errors='coerce')
tr_agg['years_since_last_transfer'] = ((today - tr_agg['last_transfer_date']).dt.days / 365)
tr_agg = tr_agg.fillna(0)


In [ ]:
#Aggregate Teammates Played With Data
tm = player_teammates_played_with.copy()
if 'minutes_played_with' in tm.columns:
    tm['minutes_played_with'] = pd.to_numeric(tm['minutes_played_with'], errors='coerce').fillna(0)
if 'joint_goal_participation' in tm.columns:
    tm['joint_goal_participation'] = pd.to_numeric(tm['joint_goal_participation'], errors='coerce').fillna(0)
tm_agg = tm.groupby('player_id').agg(
    unique_teammates = ('teammate_player_id', lambda x: x.nunique()),
    joint_minutes = ('minutes_played_with','sum') if 'minutes_played_with' in tm.columns else ('teammate_player_id','count'),
    joint_goal_participation = ('joint_goal_participation','mean') if 'joint_goal_participation' in tm.columns else ('teammate_player_id','count')
).fillna(0)
tm_agg = tm_agg.astype({'unique_teammates':'int32'}).fillna(0)


In [ ]:
#Aggregate Team Features
td = team_details.copy()
if 'club_id' in td.columns:
    td = td.rename(columns={'club_id':'team_id'})
td['team_id'] = td['team_id'].astype(str)
team_map = td.set_index('team_id')[['club_name','country_name','competition_id']].drop_duplicates()

if 'team_id' not in player_profiles.columns and 'current_club_id' in player_profiles.columns:
    player_profiles['team_id'] = player_profiles['current_club_id'].astype(str)

team_comp = team_competitions_seasons.copy()
if 'club_id' in team_comp.columns:
    team_comp['team_id'] = team_comp['club_id'].astype(str)
elif 'team_id' in team_comp.columns:
    team_comp['team_id'] = team_comp['team_id'].astype(str)
else:
    team_comp['team_id'] = np.nan

if 'season_norm' not in team_comp.columns:
    if 'season_id' in team_comp.columns:
        team_comp['season_norm'] = team_comp['season_id'].astype(str).apply(lambda x: normalize_season(x) if pd.notna(x) else np.nan)
    else:
        team_comp['season_norm'] = np.nan

team_comp_agg = team_comp.groupby('team_id').agg(
    team_seasons_count = ('season_norm','nunique'),
    competitions_count = ('competition_id','nunique'),
    latest_season = ('season_norm','max')
).fillna(0)
team_comp_agg = team_comp_agg.astype({'team_seasons_count':'int32','competitions_count':'int32'})
team_features = team_map.join(team_comp_agg, how='left').fillna(0)


In [35]:
agg_frames = {
    'perf': 'perf_agg',
    'nat': 'nat_agg',
    'inj': 'inj_agg',
    'pv': 'pv_agg',
    'tr': 'tr_agg',
    'tm': 'tm_agg',
    'team': 'team_features'
}

available = {k: globals()[v] for k,v in agg_frames.items() if v in globals()}

def prefixed_player_frame(df, prefix, idcol='player_id'):
    df2 = df.copy()
    if idcol in df2.columns:
        idx = df2[idcol].astype(str)
        df2 = df2.drop(columns=[idcol])
        df2.index = idx
    else:
        try:
            df2.index = df2.index.astype(str)
        except:
            pass
    df2 = df2.add_prefix(prefix)
    return df2

def prefixed_team_frame(df, prefix, idcol='team_id'):
    df2 = df.copy()
    if idcol in df2.columns:
        idx = df2[idcol].astype(str)
        df2 = df2.drop(columns=[idcol])
        df2.index = idx
    else:
        try:
            df2.index = df2.index.astype(str)
        except:
            pass
    df2 = df2.add_prefix(prefix)
    return df2

prefixed_frames = {}
for k,df in available.items():
    if k == 'team':
        prefixed_frames[k] = prefixed_team_frame(df, 'team_')
    else:
        prefixed_frames[k] = prefixed_player_frame(df, k + '_')

names = list(prefixed_frames.keys())
for i in range(len(names)):
    for j in range(i+1, len(names)):
        a = set(prefixed_frames[names[i]].columns)
        b = set(prefixed_frames[names[j]].columns)
        common = a.intersection(b)
        if common:
            print("OVERLAP between", names[i], "and", names[j], "->", common)

base = player_profiles.copy()
base['player_id'] = base['player_id'].astype(str)
base = base.set_index(base['player_id'])

merged = base.copy()
for k in ['inj','perf','nat','pv','tr','tm']:
    if k in prefixed_frames:
        merged = merged.join(prefixed_frames[k], how='left')

if 'player_id' in merged.columns:
    merged = merged.drop(columns=['player_id'])

merged = merged.reset_index()
if 'index' in merged.columns and 'player_id' not in merged.columns:
    merged = merged.rename(columns={'index':'player_id'})
elif 'player_id' not in merged.columns and merged.index.name:
    merged = merged.reset_index().rename(columns={merged.index.name:'player_id'})

if 'team' in prefixed_frames:
    if 'team_id' not in merged.columns and 'current_club_id' in merged.columns:
        merged['team_id'] = merged['current_club_id'].astype(str)
    if 'team_id' in merged.columns:
        merged = merged.set_index(merged['team_id'].astype(str), drop=False)
        merged = merged.join(prefixed_frames['team'], how='left')
        merged = merged.reset_index(drop=True)
    if 'player_id' not in merged.columns and merged.index.name == 'player_id':
        merged = merged.reset_index().rename(columns={'index':'player_id'})

for c in merged.select_dtypes(include=['float64','int64']).columns:
    if merged[c].isnull().any():
        merged[c] = merged[c].fillna(0)

if 'perf_minutes_played' in merged.columns and 'perf_goals' in merged.columns:
    merged['goals_per90'] = merged['perf_goals'] / (merged['perf_minutes_played']/90).replace(0,np.nan)
if 'perf_minutes_played' in merged.columns and 'perf_assists' in merged.columns:
    merged['assists_per90'] = merged['perf_assists'] / (merged['perf_minutes_played']/90).replace(0,np.nan)

for c in merged.select_dtypes(include=['int64']).columns:
    merged[c] = pd.to_numeric(merged[c], downcast='integer')
for c in merged.select_dtypes(include=['float64']).columns:
    merged[c] = pd.to_numeric(merged[c], downcast='float')
obj_cols = merged.select_dtypes(include=['object']).columns.tolist()
for c in obj_cols:
    if merged[c].nunique() < 0.5 * len(merged):
        merged[c] = merged[c].astype('category')

merged.to_csv("merged_ml.csv.gz", index=False, compression='gzip')
merged.sample(min(1000, len(merged))).to_csv("merged_ml_sample.csv", index=False)
print("merged shape:", merged.shape)
print("unique players:", merged['player_id'].nunique())


merged shape: (92671, 57)
unique players: 92671


In [36]:
df = pd.read_csv("merged_ml.csv.gz", compression="gzip")
df.head()

,player_id,player_name,date_of_birth,country_of_birth,height,citizenship,is_eu,position,main_position,foot,current_club_id,current_club_name,joined,contract_expires,player_agent_id,social_media_url_has_url,second_club_url_has_url,third_club_url_has_url,fourth_club_url_has_url,team_id,inj_total_injuries,inj_total_days_out,inj_mean_days_out,inj_seasons_with_injury,perf_minutes_played,perf_goals,perf_assists,perf_clean_sheets,perf_goals_conceded,perf_goals_per90,perf_assists_per90,nat_matches,nat_goals,pv_mv_max,pv_mv_min,pv_mv_mean,pv_mv_std,pv_mv_count,pv_mv_growth,pv_mv_current,tr_num_transfers,tr_total_transfer_fees,tr_last_transfer_fee,tr_first_transfer_date,tr_last_transfer_date,tr_years_since_last_transfer,tm_unique_teammates,tm_joint_minutes,tm_joint_goal_participation,team_club_name,team_country_name,team_competition_id,team_team_seasons_count,team_competitions_count,team_latest_season,goals_per90,assists_per90
0,1,Silvio Adzic (1),1980-09-23,Germany,0.0,Germany,True,Attack - Right Winger,Attack,NaN,123,Retired,2017-01-07,NaN,0.0,0,0,0,0,123,0.0,NaN,NaN,0.0,4552.0,69.0,5.0,0.0,0.0,1.364235,0.098858,23.0,11.0,250000.0,0.0,150000.0,1.322876e+05,3.0,-1.0,0.0,16.0,0.0,0.0,1986-01-07 00:00:00,2017-01-07 00:00:00,8.909589,0.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,1.364235,0.098858
1,100011,Éverton Silva (100011),1988-08-04,Brazil,171.0,Brazil,False,Defender - Right-Back,Defender,right,515,Without Club,2025-01-03,NaN,8373.0,0,0,0,0,515,0.0,NaN,NaN,0.0,2958.0,2.0,12.0,0.0,0.0,0.060852,0.365112,0.0,NaN,550000.0,0.0,250000.0,2.183270e+05,16.0,-1.0,0.0,18.0,200000.0,200000.0,2007-01-01 00:00:00,2025-01-03 00:00:00,0.915068,11.0,0.0,0.272727,NaN,NaN,NaN,0.0,0.0,0.0,0.060852,0.365112
2,10,Miroslav Klose (10),1978-06-09,Germany,184.0,Germany,True,Attack - Centre-Forward,Attack,right,123,Retired,2016-01-07,NaN,1126.0,1,0,0,0,123,15.0,274.0,18.266666,4.0,9417.0,258.0,138.0,0.0,0.0,2.465754,1.318891,137.0,71.0,30000000.0,0.0,9062500.0,8.164494e+06,24.0,-1.0,0.0,11.0,20000000.0,15000000.0,1993-01-07 00:00:00,2016-01-07 00:00:00,9.912329,3.0,0.0,9.666667,NaN,NaN,NaN,0.0,0.0,0.0,2.465754,1.318891
3,10001,John Thompson (10001),1981-10-12,Ireland,183.0,Ireland,True,Defender - Right-Back,Defender,both,123,Retired,2013-01-07,NaN,0.0,0,0,0,0,123,0.0,NaN,NaN,0.0,5375.0,9.0,3.0,0.0,0.0,0.150698,0.050233,14.0,0.0,325000.0,0.0,125000.0,1.750000e+05,3.0,-1.0,0.0,8.0,85000.0,85000.0,1998-01-07 00:00:00,2013-01-07 00:00:00,12.912329,1.0,0.0,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.150698,0.050233
4,100001,Carlos Auzqui (100001),1991-03-16,Argentina,180.0,Argentina,False,Attack - Right Winger,Attack,right,14554,Club Atlético Tucumán,NaN,2025-12-31,5240.0,1,0,0,0,14554,0.0,NaN,NaN,0.0,7213.0,44.0,35.0,0.0,0.0,0.549009,0.436711,0.0,NaN,3500000.0,100000.0,1479629.6,1.031453e+06,27.0,1.0,200000.0,4.0,3418000.0,2500000.0,2010-01-07 00:00:00,2022-12-02 00:00:00,3.005479,19.0,0.0,1.105263,Club Atlético Tucumán (14554),Argentina,ARGC,26.0,1.0,2025.0,0.549009,0.436711


In [37]:
merged.iloc[0].T


player_id                                           1
player_name                          Silvio Adzic (1)
date_of_birth                     1980-09-23 00:00:00
country_of_birth                              Germany
height                                            0.0
citizenship                                   Germany
is_eu                                            True
position                        Attack - Right Winger
main_position                                  Attack
foot                                              NaN
current_club_id                                   123
current_club_name                             Retired
joined                            2017-01-07 00:00:00
contract_expires                                  NaT
player_agent_id                                   0.0
social_media_url_has_url                            0
second_club_url_has_url                             0
third_club_url_has_url                              0
fourth_club_url_has_url     